# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

In [5]:
loans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   payment_history         1599 non-null   float64
 1   location_parameter      1599 non-null   float64
 2   stem_degree_score       1599 non-null   float64
 3   gpa_ranking             1599 non-null   float64
 4   alumni_success          1599 non-null   float64
 5   study_major_code        1599 non-null   float64
 6   time_to_completion      1599 non-null   float64
 7   finance_workshop_score  1599 non-null   float64
 8   cohort_ranking          1599 non-null   float64
 9   total_loan_score        1599 non-null   float64
 10  financial_aid_score     1599 non-null   float64
 11  credit_ranking          1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y[:5]

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [7]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame
X.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# Display the sizes of the training and testing datasets
print("Training set size:", X_train.shape, y_train.shape)
print("Testing set size:", X_test.shape, y_test.shape)



Training set size: (1199, 11) (1199,)
Testing set size: (400, 11) (400,)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Transform the features testing dataset using the same scaler
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [10]:
# Define the number of inputs (features) to the model
input_features = X_train_scaled.shape[1]

# Review the number of features
print("Number of input features:", input_features)

Number of input features: 11


In [11]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 64  

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 32  

# Define the number of neurons in the output layer
output_neurons = 1  



In [12]:

# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer with relu activation function
model.add(Dense(hidden_nodes_layer1, activation='relu', input_shape=(input_features,)))

# Add the second hidden layer with relu activation function
model.add(Dense(hidden_nodes_layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
# Use 'sigmoid' activation for binary classification
model.add(Dense(output_neurons, activation='sigmoid'))




In [13]:
# Display the Sequential model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                768       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,881
Trainable params: 2,881
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [14]:
# Compile the Sequential model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2)

Epoch 1/50


30/30 [==============================] - 1s 6ms/step - loss: 0.6226 - accuracy: 0.6423 - val_loss: 0.5671 - val_accuracy: 0.7333
Epoch 2/50
30/30 [==============================] - 0s 2ms/step - loss: 0.5438 - accuracy: 0.7216 - val_loss: 0.5344 - val_accuracy: 0.7375
Epoch 3/50
30/30 [==============================] - 0s 2ms/step - loss: 0.5128 - accuracy: 0.7393 - val_loss: 0.5248 - val_accuracy: 0.7500
Epoch 4/50
30/30 [==============================] - 0s 2ms/step - loss: 0.5001 - accuracy: 0.7497 - val_loss: 0.5201 - val_accuracy: 0.7542
Epoch 5/50
30/30 [==============================] - 0s 2ms/step - loss: 0.4869 - accuracy: 0.7664 - val_loss: 0.5237 - val_accuracy: 0.7583
Epoch 6/50
30/30 [==============================] - 0s 1ms/step - loss: 0.4798 - accuracy: 0.7758 - val_loss: 0.5202 - val_accuracy: 0.7500
Epoch 7/50
30/30 [==============================] - 0s 2ms/step - loss: 0.4726 - accuracy: 0.7831 - val_loss: 0.5206 - val_accuracy: 0.7542
Epoch 8/50
30/30 [=============

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [16]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print("Test loss:", loss)
print("Test accuracy:", accuracy)

13/13 [==============================] - 0s 1ms/step - loss: 0.6142 - accuracy: 0.7425
Test loss: 0.6141863465309143
Test accuracy: 0.7425000071525574


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [17]:
# Set the model's file path
model_file_path = 'saved_models/model.h5'  

# Export your model to a keras file
model.save(model_file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [18]:
from tensorflow.keras.models import load_model

# Set the model's file path
model_file_path = 'saved_models/model.h5'  # Make sure this path matches where you saved your model

# Load the model to a new object
loaded_model = load_model(model_file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [19]:
# Make predictions on the test data using the loaded model
predictions = loaded_model.predict(X_test_scaled, verbose=2)

# Display the first five predictions
print(predictions[:5])


13/13 - 0s - 79ms/epoch - 6ms/step
[[0.11781464]
 [0.42948723]
 [0.930679  ]
 [0.8825553 ]
 [0.99527127]]


In [20]:
import pandas as pd

# Assuming that 'loaded_model' is your preloaded model and 'X_test_scaled' is your scaled test dataset
# Make predictions on the testing data, with verbosity to show progress
raw_predictions = loaded_model.predict(X_test_scaled, verbose=1)

# Convert probabilities to binary results (1 if probability is 0.5 or more, 0 otherwise)
binary_predictions = (raw_predictions >= 0.5).astype(float)  # Using float to match your result format

# Create a DataFrame from the binary predictions
predictions_df = pd.DataFrame(binary_predictions, columns=['predictions'])

# Display the DataFrame
print(predictions_df)



13/13 [==============================] - 0s 843us/step
     predictions
0            0.0
1            0.0
2            1.0
3            1.0
4            1.0
..           ...
395          1.0
396          0.0
397          1.0
398          0.0
399          0.0

[400 rows x 1 columns]


### Step 4: Display a classification report with the y test data and predictions

In [21]:
# Print the classification report with the y test data and predictions
report = classification_report(y_test, binary_predictions)

# Print the classification report
print(report)


              precision    recall  f1-score   support

           0       0.72      0.75      0.73       188
           1       0.77      0.74      0.75       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.74      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. To build a recommendation system for student loan options, it's essential to collect comprehensive data that includes students' demographic, educational background, financial information, loan preferences, and behavioral data from platform interactions. This data enables personalized loan recommendations tailored to each student's unique financial needs and circumstances, ensuring the recommendations are both practical and aligned with lenders' risk assessments.**


**2. For recommending student loans, using **content-based filtering** is best because it looks closely at each student's personal information like their school, finances, and what they want in a loan. This way, each student gets loan suggestions that really fit their own needs, without mixing up what other students might like or need.**


**3. Building a recommendation system for student loans requires strict attention to **data privacy and security**, because handling sensitive financial and personal information responsibly is crucial to maintaining users' trust and complying with legal standards. Additionally, addressing **bias and fairness** is vital, as the system must fairly recommend suitable loans to all students, regardless of their background, to prevent discrimination and ensure equal financial opportunities.**

